<a href="https://colab.research.google.com/github/fasthill/ML-DL-study-alone/blob/main/5-2%20%EA%B5%90%EC%B0%A8%20%EA%B2%80%EC%A6%9D%EA%B3%BC%20%EA%B7%B8%EB%A6%AC%EB%93%9C%20%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치

## 검증 세트

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

In [4]:
directory_for_ml = '../data/data_for_ml/'
fname = 'df_sec_sel.pkl'
f_name = directory_for_ml + fname
df = pd.read_pickle(f_name)

In [5]:
data = df.iloc[:, :-2]
target = df.iloc[:, -1]

In [11]:
rannum = 42
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=rannum, stratify=target)

In [12]:
sts = StandardScaler()
sts.fit(train_input)
train_scaled = sts.transform(train_input)
test_scaled = sts.transform(test_input)

훈련세트가 2만이 넘는다면 훈련세트로만 전체를 사용하고 val set는 없어도 되고, 데이터가 충분히 많다면 val set는 5~ 10%만 지정해서 사용해도 됨

In [13]:
print(train_input.shape, test_input.shape)

(127, 18) (32, 18)


In [14]:
dt = DecisionTreeClassifier(random_state=rannum)
dt.fit(train_scaled, train_target)

DecisionTreeClassifier(random_state=42)

In [15]:
train_score = dt.score(train_scaled, train_target)
test_score = dt.score(test_scaled, test_target)
print('train_score : {} \ntest_score : {}'.format(train_score, test_score))

train_score : 1.0 
test_score : 0.8125


## 교차 검증 <br>
ML에서는 샘플수가 충분히 많지 않기 때문에 교차검증을 시행하여 최적의 파마미터을 찾게 되지만, 딥러닝에서는 샘플 수가 충분하여 교차검증을 하지 않고, 훈련세트와 검증세트로만 나누어 사용함 <br>
따라서, tensorflow, keras같은 경우는 gridsearchcv와 같은 기능이 그렇게 많지 않음. tensorflow, keras는 데이터을 읽어들이는 도구들이 많이 발달되어 있음.

In [16]:
from sklearn.model_selection import cross_validate

In [17]:
dt = DecisionTreeClassifier(random_state=rannum)

In [18]:
scores = cross_validate(dt,
                        X = train_scaled, y=train_target,
                        cv=40, n_jobs=multiprocessing.cpu_count(),
                        verbose=1)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  40 out of  40 | elapsed:    1.7s finished


In [19]:
scores

{'fit_time': array([0.00299144, 0.00199485, 0.00199461, 0.00199509, 0.00298762,
        0.00199461, 0.00199509, 0.00299191, 0.00199556, 0.0029912 ,
        0.00199437, 0.00199485, 0.00199485, 0.00199413, 0.00199461,
        0.00199461, 0.00199485, 0.0009973 , 0.00099778, 0.00199485,
        0.00099802, 0.00099707, 0.00199413, 0.00199485, 0.00199485,
        0.00199509, 0.00199437, 0.00199485, 0.00199437, 0.00199509,
        0.00199366, 0.00099754, 0.00199389, 0.0009973 , 0.00199461,
        0.00199437, 0.00199437, 0.0009973 , 0.00099707, 0.00199533]),
 'score_time': array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.00099802, 0.00099683, 0.        , 0.00099683, 0.00099778,
        0.00099778, 0.00099707, 0.        , 0.        , 0.        ,
        0.0009973 , 0.        , 0.00099778, 0.00099683, 0.00099707,
        0.00099707, 0.00099707, 0.00099778, 0.        , 0.0009973 ,
        0.00099707, 0.00099778, 0.        , 0.00099874, 0.        ,
        0.        , 

In [20]:
print("test score : {} (+/- {}".format(scores['test_score'].mean(), scores['test_score'].std()))

test score : 0.7208333333333332 (+/- 0.232401412789739


'# dt.score(test_input, test_target) <br>
cross_validate에서의 fit기능은 무엇인가? <br>
.fit는 언제하는가?

#### 분류기(splitter) 사용 <br>
클라스가 고르게 섞이게 하기 위하여 사용 <br>
회귀모델에서는 KFold() 객체, 분류모델에서는 StratifiedKFold() 사용 <br>
cross_validate method에서는 estimator에 따라 분류인지, 회귀인지를 확인하고, cv를 StratifiedKFlod()를 사용할지 KFold를 사용할 지 자동으로 결정하여 진행함.

In [21]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt,
                        X = train_scaled, y=train_target,
                        cv=StratifiedKFold(30), 
                        n_jobs=multiprocessing.cpu_count(),
                        verbose=1)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  30 out of  30 | elapsed:    0.0s finished


In [22]:
print("test score : {} (+/- {}".format(scores['test_score'].mean(), scores['test_score'].std()))

test score : 0.6916666666666667 (+/- 0.21137775558358918


위에서 cv=5를 적용한 cross_validate결과와 동일한 결과가 나옴.<br>
아래 splitter를 사용하여 상세하게 파라미터를 구성하여 사용

In [23]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=rannum)
scores = cross_validate(dt, train_scaled, train_target, cv=splitter)
print("test score : {} (+/- {}".format(scores['test_score'].mean(), scores['test_score'].std()))

test score : 0.7076923076923077 (+/- 0.11243377771881367


## 하이퍼파라미터 튜닝 <br>
최적의 파라미터를 찾기 위한 loop실행

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [22]:
# dt = DecisionTreeClassifier(random_state=42)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                  params,
                  n_jobs=-1) # -1:모든 cpu core를 사용
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [23]:
dt = gs.best_estimator_
dt_score = dt.score(train_input, train_target)
print('best estimator score: {}'.format(dt_score))

best estimator score: 1.0


In [24]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [25]:
for k, v in gs.cv_results_.items():
  if k == 'params':
    print('params')
    for k, v in params.items():
      print(k, v)
    continue
  print('{}: {}'.format(k, v))

mean_fit_time: [0.00299287 0.00299191 0.00239329 0.0026     0.0029973 ]
std_fit_time: [2.61174468e-07 4.26496120e-07 4.88188328e-04 4.92771566e-04
 2.41218340e-05]
mean_score_time: [0.00099707 0.0015955  0.00119677 0.00139575 0.00100288]
std_score_time: [4.76837158e-07 4.88422116e-04 3.99041187e-04 4.88360648e-04
 1.27437797e-05]
param_min_impurity_decrease: [0.0001 0.0002 0.0003 0.0004 0.0005]
params
min_impurity_decrease [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]
split0_test_score: [0.61538462 0.61538462 0.61538462 0.61538462 0.61538462]
split1_test_score: [0.5 0.5 0.5 0.5 0.5]
split2_test_score: [0.52 0.52 0.52 0.52 0.52]
split3_test_score: [0.64 0.64 0.64 0.64 0.64]
split4_test_score: [0.72 0.72 0.72 0.72 0.72]
mean_test_score: [0.59907692 0.59907692 0.59907692 0.59907692 0.59907692]
std_test_score: [0.08078666 0.08078666 0.08078666 0.08078666 0.08078666]
rank_test_score: [1 1 1 1 1]


In [26]:
gs.cv_results_['mean_test_score']

array([0.59907692, 0.59907692, 0.59907692, 0.59907692, 0.59907692])

In [36]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [37]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [38]:
print(gs.best_params_)

{'max_depth': 5, 'min_impurity_decrease': 0.0001, 'min_samples_split': 82}


In [39]:
print(np.max(gs.cv_results_['mean_test_score']))

0.42400000000000004


### 랜덤 서치

In [40]:
from scipy.stats import uniform, randint
# from numpy.random import uniform, randint # 기능은 같고 모양은 틀리나, 객체를 생성하지 못함.(아래 예 참조)

In [41]:
rgen = randint(0, 10)
rgen.rvs(10)

array([9, 9, 9, 8, 4, 1, 0, 5, 0, 2], dtype=int64)

In [42]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([100, 101, 101,  97,  95, 114,  96,  98,  82, 116], dtype=int64))

In [43]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.81857619, 0.60104731, 0.0774513 , 0.83074643, 0.14811319,
       0.86239832, 0.02531275, 0.89304147, 0.9532159 , 0.52481687])

In [44]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [45]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42) #n_iter: params 의 임의변수를 100번 시행
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000020C22D03F70>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000020C22D03610>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000020C22D00820>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000020C22D037C0>},
                   random_state=42)

In [46]:
print(gs.best_params_)

{'max_depth': 31, 'min_impurity_decrease': 0.0005165099478703662, 'min_samples_leaf': 11, 'min_samples_split': 24}


In [47]:
print(np.max(gs.cv_results_['mean_test_score']))

0.44800000000000006


In [48]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.3125


## 확인문제

In [ ]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

In [ ]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))